In [3]:
import pandas as pd
import anthropic
from anthropic import Anthropic, HUMAN_PROMPT, AI_PROMPT
import os

In [4]:
print(anthropic.__version__)

0.3.11


In [5]:
pd.read_csv('your_file.csv')

,Prompt
0,realistic car 3 d render sci - fi car and sci ...
1,a comic potrait of a female necromamcer with b...
2,"steampunk market interior, colorful, 3 d scene..."
3,"“A portrait of a cyborg in a golden suit, D&D ..."
4,A full portrait of a beautiful post apocalypti...
...,...
73713,ismail inceoglu epic oil on canvas painting of...
73714,eating crayons and being reborn in the loving ...
73715,"ilya kuvshinov with long hair, sky blue hair, ..."
73716,cyberpunk woman with green hair wearing futuri...


In [6]:
df_hqprompts = pd.read_csv('your_file.csv')

In [7]:
df_hqprompts.head(5)

,Prompt
0,realistic car 3 d render sci - fi car and sci ...
1,a comic potrait of a female necromamcer with b...
2,"steampunk market interior, colorful, 3 d scene..."
3,"“A portrait of a cyborg in a golden suit, D&D ..."
4,A full portrait of a beautiful post apocalypti...


In [8]:
df_hqprompts_truncated = df_hqprompts[:15000]

In [9]:
df_hqprompts_truncated

,Prompt
0,realistic car 3 d render sci - fi car and sci ...
1,a comic potrait of a female necromamcer with b...
2,"steampunk market interior, colorful, 3 d scene..."
3,"“A portrait of a cyborg in a golden suit, D&D ..."
4,A full portrait of a beautiful post apocalypti...
...,...
14995,symmetry!! portrait of a silhouette of a vikin...
14996,"ultra realistic illustration, bill murray in g..."
14997,"Neil Young design, character sheet, kim jung g..."
14998,ange ushiromiya battler ange ushiromiya ryona!...


## Anthropic prompting with Claude

In [10]:
anthropic = Anthropic(
api_key = os.environ["ANTHROPIC_API_KEY_ORG"]
)

In [11]:
HUMAN_PROMPT

'\n\nHuman:'

In [12]:
AI_PROMPT

'\n\nAssistant:'

In [13]:
prompt_head = """
<context>
A great prompt contains specific keywords and details that provide clear direction for AI image generation. A simple prompt conveys the overall idea in a general descriptive way.

Your task is to take advanced, high-quality prompts I provide, and simplify them into beginner prompt versions. This is to create training data for fine-tuning a model that can take user’s basic prompts and create high quality prompts. Your role is to take my advanced prompts as input (advancedPrompt), and simplify them into beginner versions (simplePrompt). You will do this by adopting the mindset of an enthusiastic but inexperienced user who describes images in a non-specialized yet descriptive way.
You will aim to generate simple, beginner prompts that retain the overall essence of what the user intends, while omitting stylistic specifics.

You need to provide the prompts in this JSON format:
{ "simplePrompt": "", "advancedPrompt": "" }
Here are some examples:
<examples>
{"simplePrompt":"Painting of cute pink-haired anime girl holding magic wand","advancedPrompt":"Portrait of a magical anime girl with long pink hair, big sparkling eyes, frilly pink dress, holding a star wand, delicate makeup, soft studio lighting, highly detailed digital painting"}
{"simplePrompt":"Realistic photo of Roger Federer serving ball in stadium","advancedPrompt":"Photorealistic image of Roger Federer serving on Wimbledon center court, intricate details on tennis clothes and equipment, camera angle from the side, cinematic lighting, 8K resolution"}
{"simplePrompt":"Drawing of an astronaut in detailed space suit floating near Saturn and Jupiter","advancedPrompt":"An astronaut floating in space near Saturn and Jupiter, intricate NASA spacesuit, oxygen tanks and gear, awe-inspiring perspective, astrophotography, National Geographic style"}
{"simplePrompt":"Photo of a delicious bowl of hot ramen noodles with pork slices","advancedPrompt":"Bowl of hot, fresh, mouthwatering ramen noodles, extra rich creamy broth, slices of savory pork, green onions as garnish, shallow depth of field, image by David Jakab photography"}
{"simplePrompt":"Cowboy on galloping horse riding through Rocky mountains","advancedPrompt":"A rugged cowboy riding a galloping horse through the Rocky Mountains, cinematic backlighting, misty morning light, detailed wilderness landscape, Western movie poster style"}
</examples>

When producing the output in json format, do not use numbering for each json object. Separate them only using a newline.
To reiterate, here are the instructions that you need to follow for generating simple prompts:
<li>Prompts need to be between 10-15 words </li>
<li>Provides clear direction with straightforward vocabulary</li>
<li>Conveys core idea in plain language</li>
<li>Retains only most important key details</li>

</context>

REMEMBER, simple prompts need to have sufficient nuance exhibiting the enthusiasm of a begineer! Now here are 50 advanced prompts, please complete the task as mentioned above by generating simple prompts in the mentioned json format. 
ADVANCED PROMPTS:
"""

In [15]:
output_anchor = 'Here are the simple prompts in JSON format:\n\n'

In [14]:
HUMAN_PROMPT + prompt_head + AI_PROMPT

'\n\nHuman:\n<context>\nA great prompt contains specific keywords and details that provide clear direction for AI image generation. A simple prompt conveys the overall idea in a general descriptive way.\n\nYour task is to take advanced, high-quality prompts I provide, and simplify them into beginner prompt versions. This is to create training data for fine-tuning a model that can take user’s basic prompts and create high quality prompts. Your role is to take my advanced prompts as input (advancedPrompt), and simplify them into beginner versions (simplePrompt). You will do this by adopting the mindset of an enthusiastic but inexperienced user who describes images in a non-specialized yet descriptive way.\nYou will aim to generate simple, beginner prompts that retain the overall essence of what the user intends, while omitting stylistic specifics.\n\nYou need to provide the prompts in this JSON format:\n{ "simplePrompt": "", "advancedPrompt": "" }\nHere are some examples:\n<examples>\n{"

1. Take the prompt structure
2. Loop over the 15k prompts
3. Add 50 prompts at once
4. Generate the 50 json outputs
5. Append them to a json file
6. Run a new session by resetting the prompt, and performing steps 3-5

In [25]:
fiftyyy_prompts = ""
for index,prompt_item in enumerate(df_hqprompts_truncated[300:350]['Prompt'].astype(str).tolist()):
    fiftyyy_prompts += f"""\n{index+1}. {prompt_item}"""
anthropic.count_tokens(fiftyyy_prompts),anthropic.count_tokens(f"""{HUMAN_PROMPT}{prompt_head}{fiftyyy_prompts}{AI_PROMPT}{output_anchor}""")

(3142, 3835)

<h3>We notice that batching prompts into batches of 50 means they won't fit within the 4096 token input limit of Claude.</h3>

<h4>That's why our previous pipeline failed on the fourth batch for 150:200 prompts batch</h4>

In [36]:
##Iterating over the entire dataset to retrieve batches of 50 prompts
for batch_number in range(15000//50):
    fifty_prompts = ""
    for index,prompt_item in enumerate(df_hqprompts_truncated[batch_number*50:(batch_number+1)*50]['Prompt'].astype(str).tolist()):
        fifty_prompts += f"""\n{index+1}. {prompt_item}"""
        batch_range = [batch_number*50,(batch_number+1)*50]
    concat_prompt = f"""{HUMAN_PROMPT}{prompt_head}{fifty_prompts}{AI_PROMPT}{output_anchor}"""
#         print(f"""For batch {batch_range}, number of tokens are {anthropic.count_tokens({HUMAN_PROMPT}{prompt_head}{fifty_prompts}{AI_PROMPT}{output_anchor})}""")
    print(f"""For batch {batch_range}, number of tokens are {anthropic.count_tokens(concat_prompt)}""")


For batch [0, 50], number of tokens are 3671
For batch [50, 100], number of tokens are 3904
For batch [100, 150], number of tokens are 3786
For batch [150, 200], number of tokens are 4141
For batch [200, 250], number of tokens are 4147
For batch [250, 300], number of tokens are 3934
For batch [300, 350], number of tokens are 3835
For batch [350, 400], number of tokens are 4156
For batch [400, 450], number of tokens are 3797
For batch [450, 500], number of tokens are 3900
For batch [500, 550], number of tokens are 3691
For batch [550, 600], number of tokens are 3853
For batch [600, 650], number of tokens are 4047
For batch [650, 700], number of tokens are 4075
For batch [700, 750], number of tokens are 4066
For batch [750, 800], number of tokens are 4023
For batch [800, 850], number of tokens are 4182
For batch [850, 900], number of tokens are 3836
For batch [900, 950], number of tokens are 4135
For batch [950, 1000], number of tokens are 3884
For batch [1000, 1050], number of tokens ar

For batch [12100, 12150], number of tokens are 4155
For batch [12150, 12200], number of tokens are 4366
For batch [12200, 12250], number of tokens are 3779
For batch [12250, 12300], number of tokens are 4086
For batch [12300, 12350], number of tokens are 3973
For batch [12350, 12400], number of tokens are 3943
For batch [12400, 12450], number of tokens are 4237
For batch [12450, 12500], number of tokens are 4036
For batch [12500, 12550], number of tokens are 4230
For batch [12550, 12600], number of tokens are 4028
For batch [12600, 12650], number of tokens are 3631
For batch [12650, 12700], number of tokens are 3838
For batch [12700, 12750], number of tokens are 4018
For batch [12750, 12800], number of tokens are 3996
For batch [12800, 12850], number of tokens are 4209
For batch [12850, 12900], number of tokens are 4010
For batch [12900, 12950], number of tokens are 3930
For batch [12950, 13000], number of tokens are 4225
For batch [13000, 13050], number of tokens are 3846
For batch [1

### We batch prompts into batches of 40 so they fit within the 4096 token input limit of Claude

In [41]:
##Iterating over the entire dataset to retrieve batches of 50 prompts
for batch_number in range(15000//40):
    fifty_prompts = ""
    for index,prompt_item in enumerate(df_hqprompts_truncated[batch_number*40:(batch_number+1)*40]['Prompt'].astype(str).tolist()):
        fifty_prompts += f"""\n{index+1}. {prompt_item}"""
        batch_range = [batch_number*40,(batch_number+1)*40]
    concat_prompt = f"""{HUMAN_PROMPT}{prompt_head}{fifty_prompts}{AI_PROMPT}{output_anchor}"""
#         print(f"""For batch {batch_range}, number of tokens are {anthropic.count_tokens({HUMAN_PROMPT}{prompt_head}{fifty_prompts}{AI_PROMPT}{output_anchor})}""")
    print(f"""For batch {batch_range}, number of tokens are {anthropic.count_tokens(concat_prompt)}""")


For batch [0, 40], number of tokens are 3099
For batch [40, 80], number of tokens are 3265
For batch [80, 120], number of tokens are 3060
For batch [120, 160], number of tokens are 3380
For batch [160, 200], number of tokens are 3391
For batch [200, 240], number of tokens are 3459
For batch [240, 280], number of tokens are 3373
For batch [280, 320], number of tokens are 3196
For batch [320, 360], number of tokens are 3266
For batch [360, 400], number of tokens are 3471
For batch [400, 440], number of tokens are 3177
For batch [440, 480], number of tokens are 3181
For batch [480, 520], number of tokens are 3272
For batch [520, 560], number of tokens are 3120
For batch [560, 600], number of tokens are 3184
For batch [600, 640], number of tokens are 3323
For batch [640, 680], number of tokens are 3486
For batch [680, 720], number of tokens are 3322
For batch [720, 760], number of tokens are 3365
For batch [760, 800], number of tokens are 3408
For batch [800, 840], number of tokens are 346

In [42]:
for batch_number in range(15000//50):
    fifty_prompts = ""
    for index,prompt_item in enumerate(df_hqprompts_truncated[batch_number*50:(batch_number+1)*50]['Prompt'].astype(str).tolist()):
        fifty_prompts += f"""\n{index+1}. {prompt_item}"""
        batch_range = [batch_number*50,(batch_number+1)*50]
    concat_prompt = f"""{HUMAN_PROMPT}{prompt_head}{fifty_prompts}{AI_PROMPT}{output_anchor}"""
    if anthropic.count_tokens(concat_prompt)>4096:
        print(f"""For batch {batch_range}, number of tokens are {anthropic.count_tokens(concat_prompt)}""")


For batch [150, 200], number of tokens are 4141
For batch [200, 250], number of tokens are 4147
For batch [350, 400], number of tokens are 4156
For batch [800, 850], number of tokens are 4182
For batch [900, 950], number of tokens are 4135
For batch [1100, 1150], number of tokens are 4156
For batch [1200, 1250], number of tokens are 4230
For batch [1650, 1700], number of tokens are 4170
For batch [1700, 1750], number of tokens are 4131
For batch [1750, 1800], number of tokens are 4215
For batch [1900, 1950], number of tokens are 4177
For batch [2300, 2350], number of tokens are 4226
For batch [2450, 2500], number of tokens are 4137
For batch [2550, 2600], number of tokens are 4551
For batch [2600, 2650], number of tokens are 4190
For batch [2700, 2750], number of tokens are 4192
For batch [2750, 2800], number of tokens are 4229
For batch [3200, 3250], number of tokens are 4157
For batch [3350, 3400], number of tokens are 4108
For batch [3450, 3500], number of tokens are 4103
For batch 

In [16]:
##Iterating over the entire dataset to retrieve batches of 50 prompts
for batch_number in range(50//50):
    fifty_prompts = ""
    for index,prompt_item in enumerate(df_hqprompts_truncated[batch_number*50:(batch_number+1)*50]['Prompt'].astype(str).tolist()):
        fifty_prompts += f"""\n{index+1}. {prompt_item}"""
    print(fifty_prompts)


1. realistic car 3 d render sci - fi car and sci - fi robotic factory structure in the coronation of napoleon painting and digital billboard with point cloud in the middle, unreal engine 5, keyshot, octane, artstation trending, ultra high detail, ultra realistic, cinematic, 8 k, 1 6 k, in style of zaha hadid, in style of nanospace michael menzelincev, in style of lee souder, in plastic, dark atmosphere, tilt shift, depth of field,
2. a comic potrait of a female necromamcer with big and cute eyes, fine - face, realistic shaded perfect face, fine details. night setting. very anime style. realistic shaded lighting poster by ilya kuvshinov katsuhiro, magali villeneuve, artgerm, jeremy lipkin and michael garmash, rob rey and kentaro miura style, trending on art station
3. steampunk market interior, colorful, 3 d scene, greg rutkowski, zabrocki, karlkka, jayison devadas, trending on artstation, 8 k, ultra wide angle, zenith view, pincushion lens effect
4. “A portrait of a cyborg in a golden

In [17]:
print(f"""{HUMAN_PROMPT}{prompt_head}{fifty_prompts}{AI_PROMPT}{output_anchor}""")




Human:
<context>
A great prompt contains specific keywords and details that provide clear direction for AI image generation. A simple prompt conveys the overall idea in a general descriptive way.

Your task is to take advanced, high-quality prompts I provide, and simplify them into beginner prompt versions. This is to create training data for fine-tuning a model that can take user’s basic prompts and create high quality prompts. Your role is to take my advanced prompts as input (advancedPrompt), and simplify them into beginner versions (simplePrompt). You will do this by adopting the mindset of an enthusiastic but inexperienced user who describes images in a non-specialized yet descriptive way.
You will aim to generate simple, beginner prompts that retain the overall essence of what the user intends, while omitting stylistic specifics.

You need to provide the prompts in this JSON format:
{ "simplePrompt": "", "advancedPrompt": "" }
Here are some examples:
<examples>
{"simplePrompt":

### Using Anthropic Tokenizer

In [19]:
client = Anthropic()

In [20]:
client.count_tokens(f"""{HUMAN_PROMPT}{prompt_head}{fifty_prompts}{AI_PROMPT}{output_anchor}""")

3671

### Testing by iterating over first 50 examples + prompting Claude

### Note:  You cannot run inference on 150 input prompts at once because Claude gets capped at 4096 tokens for output. 50 inputs is the sweet spot because they range 3000-4000 tokens</h4>

In [21]:
df_hqprompts_truncated.shape

(15000, 1)

In [22]:
##Pricing for Anthropic if I build a dataset of 15k examples
(32.68*1200000/1000000) + (32.68*1200000/1000000)

78.432

In [187]:
print(f"""{HUMAN_PROMPT}{prompt_head}{fifty_prompts}{AI_PROMPT}{output_anchor}""")



Human:
<context>
A great prompt contains specific keywords and details that provide clear direction for AI image generation models. A simple prompt conveys the overall idea in a general descriptive way.
Your task is to take high-quality prompts I provide, and simplify them into beginner prompt versions. This is to create training data for fine-tuning a model that can take user’s basic prompts and create high quality prompts.
Your role is to take my high-quality prompts as input (hqPrompt), and simplify them into beginner versions (simplePrompt). You will do this by adopting the mindset of an enthusiastic but inexperienced user who describes images in a non-specialized yet descriptive way.
You will aim to generate beginner prompts that retain the overall essence of what the user intends, while omitting stylistic specifics. Each prompt you generate need to be 10-15 words.


You need to provide the prompts in this JSON format:
{ "simplePrompt": "", "hqPrompt": "" }
Here are some example

#### The string object that will hold all our json objects

In [24]:
prompts_dataset_json = ''

## Building the pipeline for creating the synthetic dataset

In [25]:
# Poll the 15k example dataset
for batch_number in range(5000//50):
    
    fifty_prompts = ""
    
    # Take in 50 prompts
    for index,prompt_item in enumerate(df_hqprompts_truncated[batch_number*50:(batch_number+1)*50]['Prompt'].astype(str).tolist()):
        
        # Append them to prompt body
        fifty_prompts += f"""\n{index+1}. {prompt_item}"""
        
    # Run this appended prompt to Claude
    response = anthropic.completions.create(
        model="claude-2.0",
        max_tokens_to_sample=50000,
        prompt=f"""{HUMAN_PROMPT}{prompt_head}{fifty_prompts}{AI_PROMPT}{output_anchor}""",
        )
    full_response_50 = response.completion
    
    # Append them to the string object prompts_dataset_json
    prompts_dataset_json+=full_response_50
    
    
    response = anthropic.completions.create(
        model="claude-2.0",
        max_tokens_to_sample=50000,
        prompt=f"""{HUMAN_PROMPT}{prompt_head}{fifty_prompts}{AI_PROMPT}{output_anchor}""",
        )

with open("prompt_pairs.txt", "w") as file:
    file.write(prompts_dataset_json)

InternalServerError: upstream request timeout

In [28]:
with open("prompt_pairs.txt", "w") as file:
    file.write(prompts_dataset_json)

In [27]:
fifty_prompts

'\n1. a man and an alien ship in the form of a glowing sphere, dark forest, fog, rain, volumetric lighting, beautiful, mystique, golden hour, sharp focus, ultra detailed, concept art, gilles beloeil, andy park, jan urschel, dylan cole, noir art house, 4 k, 3 5 mm, fujifilm\n2. angry character, Moebius, Kim Jung Gi, Greg Rutkowski, Zabrocki, Karlkka, Jayison Devadas, Phuoc Quan, trending on Artstation, 8K, ultra wide angle, pincushion lens effect\n3. a highly detailed byzantine painting of scarlett johansson and emma watson as red haired queens sharing a soft kiss under a waterfall in a gossamer purple dress, epic fantasy, viewed in profile from far away, ultrawide lens, art by artgerm and greg rutkowski and alphonse mucha, volumetric lighting, 4 k resolution, trending on artstation, masterpiece\n4. concept art by craig mullins astronaut in futuristic dark and empty spaceship. infrared complex and hyperdetailed technical suit design. mandelbulb fractal underwater space. reflection and d

In [26]:
print(prompts_dataset_json)


{"simplePrompt":"Anime girl with pink hair and frilly dress holding magic wand","advancedPrompt":"Portrait of a magical anime girl with long pink hair, big sparkling eyes, frilly pink dress, holding a star wand, delicate makeup, soft studio lighting, highly detailed digital painting"}
{"simplePrompt":"Roger Federer serving tennis ball in stadium","advancedPrompt":"Photorealistic image of Roger Federer serving on Wimbledon center court, intricate details on tennis clothes and equipment, camera angle from the side, cinematic lighting, 8K resolution"}  
{"simplePrompt":"Astronaut floating in space near Saturn and Jupiter","advancedPrompt":"An astronaut floating in space near Saturn and Jupiter, intricate NASA spacesuit, oxygen tanks and gear, awe-inspiring perspective, astrophotography, National Geographic style"}
{"simplePrompt":"Bowl of delicious ramen noodles with pork slices","advancedPrompt":"Bowl of hot, fresh, mouthwatering ramen noodles, extra rich creamy broth, slices of savory 

In [209]:
#Testing by extracting 5 prompts
five_prompts = ""
for i,j in enumerate(df_hqprompts_truncated[0:5]['Prompt'].astype(str).tolist()):
    five_prompts += f"""\n{i+1}. {j}"""
print(five_prompts)


1. realistic car 3 d render sci - fi car and sci - fi robotic factory structure in the coronation of napoleon painting and digital billboard with point cloud in the middle, unreal engine 5, keyshot, octane, artstation trending, ultra high detail, ultra realistic, cinematic, 8 k, 1 6 k, in style of zaha hadid, in style of nanospace michael menzelincev, in style of lee souder, in plastic, dark atmosphere, tilt shift, depth of field,
2. a comic potrait of a female necromamcer with big and cute eyes, fine - face, realistic shaded perfect face, fine details. night setting. very anime style. realistic shaded lighting poster by ilya kuvshinov katsuhiro, magali villeneuve, artgerm, jeremy lipkin and michael garmash, rob rey and kentaro miura style, trending on art station
3. steampunk market interior, colorful, 3 d scene, greg rutkowski, zabrocki, karlkka, jayison devadas, trending on artstation, 8 k, ultra wide angle, zenith view, pincushion lens effect
4. “A portrait of a cyborg in a golden

In [188]:
client.count_tokens(f"""{HUMAN_PROMPT}{prompt_head}{fifty_prompts}{AI_PROMPT}{output_anchor}""")

1062

In [193]:
response = anthropic.completions.create(
        model="claude-2.0",
        max_tokens_to_sample=50000,
        prompt=f"""{HUMAN_PROMPT}{prompt_head}{fifty_prompts}{AI_PROMPT}{output_anchor}""",
        )
full_response_10 = response.completion

In [190]:
client.count_tokens(full_response_10)

456

In [194]:
prompts_dataset_json = ''
prompts_dataset_json+full_response_10

'\n{"simplePrompt":"Rendering of futuristic sci-fi car in robotic factory","hqPrompt":"realistic car 3 d render sci - fi car and sci - fi robotic factory structure in the coronation of napoleon painting and digital billboard with point cloud in the middle, unreal engine 5, keyshot, octane, artstation trending, ultra high detail, ultra realistic, cinematic, 8 k, 1 6 k, in style of zaha hadid, in style of nanospace michael menzelincev, in style of lee souder, in plastic, dark atmosphere, tilt shift, depth of field,"}\n\n{"simplePrompt":"Portrait of female necromancer with big cute eyes in anime style","hqPrompt":"a comic potrait of a female necromamcer with big and cute eyes, fine - face, realistic shaded perfect face, fine details. night setting. very anime style. realistic shaded lighting poster by ilya kuvshinov katsuhiro, magali villeneuve, artgerm, jeremy lipkin and michael garmash, rob rey and kentaro miura style, trending on art station"}\n\n{"simplePrompt":"Steampunk market inter

In [195]:
print(full_response_10)


{"simplePrompt":"Rendering of futuristic sci-fi car in robotic factory","hqPrompt":"realistic car 3 d render sci - fi car and sci - fi robotic factory structure in the coronation of napoleon painting and digital billboard with point cloud in the middle, unreal engine 5, keyshot, octane, artstation trending, ultra high detail, ultra realistic, cinematic, 8 k, 1 6 k, in style of zaha hadid, in style of nanospace michael menzelincev, in style of lee souder, in plastic, dark atmosphere, tilt shift, depth of field,"}

{"simplePrompt":"Portrait of female necromancer with big cute eyes in anime style","hqPrompt":"a comic potrait of a female necromamcer with big and cute eyes, fine - face, realistic shaded perfect face, fine details. night setting. very anime style. realistic shaded lighting poster by ilya kuvshinov katsuhiro, magali villeneuve, artgerm, jeremy lipkin and michael garmash, rob rey and kentaro miura style, trending on art station"}

{"simplePrompt":"Steampunk market interior sc

In [135]:
client.count_tokens(full_response)

3732

In [142]:
print(full_response)

 Here are 50 simplified prompts in JSON format for the provided high-quality prompts:

{"simplePrompt":"Painting of futuristic car near robot factory and art billboard","hqPrompt":"realistic car 3 d render sci - fi car and sci - fi robotic factory structure in the coronation of napoleon painting and digital billboard with point cloud in the middle, unreal engine 5, keyshot, octane, artstation trending, ultra high detail, ultra realistic, cinematic, 8 k, 1 6 k, in style of zaha hadid, in style of nanospace michael menzelincev, in style of lee souder, in plastic, dark atmosphere, tilt shift, depth of field,"}

{"simplePrompt":"Comic portrait of cute anime girl necromancer","hqPrompt":"a comic potrait of a female necromamcer with big and cute eyes, fine - face, realistic shaded perfect face, fine details. night setting. very anime style. realistic shaded lighting poster by ilya kuvshinov katsuhiro, magali villeneuve, artgerm, jeremy lipkin and michael garmash, rob rey and kentaro miura st

In [125]:
print(full_response)

 Here are 5 simplified beginner prompts for the high quality prompts you provided:

1. {"simplePrompt":"Digital painting of futuristic car in sci-fi factory with digital billboard","hqPrompt":"realistic car 3 d render sci - fi car and sci - fi robotic factory structure in the coronation of napoleon painting and digital billboard with point cloud in the middle, unreal engine 5, keyshot, octane, artstation trending, ultra high detail, ultra realistic, cinematic, 8 k, 1 6 k, in style of zaha hadid, in style of nanospace michael menzelincev, in style of lee souder, in plastic, dark atmosphere, tilt shift, depth of field,"}

2. {"simplePrompt":"Anime style portrait of cute female necromancer","hqPrompt":"a comic potrait of a female necromamcer with big and cute eyes, fine - face, realistic shaded perfect face, fine details. night setting. very anime style. realistic shaded lighting poster by ilya kuvshinov katsuhiro, magali villeneuve, artgerm, jeremy lipkin and michael garmash, rob rey and

## Way to iterate over all 15000 examples

In [ ]:
list_of_examples = ""
for batch in range(15000//100):
    for index,item in enumerate(df_hqprompts_truncated[batch*50:(batch+1)*50]['Prompt'].astype(str).tolist()):
        list_of_examples += f"""{index}. {item}\n\n"""

In [54]:
print(empty_string)

0. realistic car 3 d render sci - fi car and sci - fi robotic factory structure in the coronation of napoleon painting and digital billboard with point cloud in the middle, unreal engine 5, keyshot, octane, artstation trending, ultra high detail, ultra realistic, cinematic, 8 k, 1 6 k, in style of zaha hadid, in style of nanospace michael menzelincev, in style of lee souder, in plastic, dark atmosphere, tilt shift, depth of field,

1. a comic potrait of a female necromamcer with big and cute eyes, fine - face, realistic shaded perfect face, fine details. night setting. very anime style. realistic shaded lighting poster by ilya kuvshinov katsuhiro, magali villeneuve, artgerm, jeremy lipkin and michael garmash, rob rey and kentaro miura style, trending on art station

2. steampunk market interior, colorful, 3 d scene, greg rutkowski, zabrocki, karlkka, jayison devadas, trending on artstation, 8 k, ultra wide angle, zenith view, pincushion lens effect

3. “A portrait of a cyborg in a gold

In [19]:
input_for_model = ""
for i,j in enumerate(empty_string):
    input_for_model+=

In [20]:
input_for_model

"1.['realistic car 3 d render sci - fi car and sci - fi robotic factory structure in the coronation of napoleon painting and digital billboard with point cloud in the middle, unreal engine 5, keyshot, octane, artstation trending, ultra high detail, ultra realistic, cinematic, 8 k, 1 6 k, in style of zaha hadid, in style of nanospace michael menzelincev, in style of lee souder, in plastic, dark atmosphere, tilt shift, depth of field,']\n2.['a comic potrait of a female necromamcer with big and cute eyes, fine - face, realistic shaded perfect face, fine details. night setting. very anime style. realistic shaded lighting poster by ilya kuvshinov katsuhiro, magali villeneuve, artgerm, jeremy lipkin and michael garmash, rob rey and kentaro miura style, trending on art station']\n3.['steampunk market interior, colorful, 3 d scene, greg rutkowski, zabrocki, karlkka, jayison devadas, trending on artstation, 8 k, ultra wide angle, zenith view, pincushion lens effect']\n4.['“A portrait of a cybor

In [78]:
df_hqprompts_truncated[:50].values

array([['realistic car 3 d render sci - fi car and sci - fi robotic factory structure in the coronation of napoleon painting and digital billboard with point cloud in the middle, unreal engine 5, keyshot, octane, artstation trending, ultra high detail, ultra realistic, cinematic, 8 k, 1 6 k, in style of zaha hadid, in style of nanospace michael menzelincev, in style of lee souder, in plastic, dark atmosphere, tilt shift, depth of field,'],
       ['a comic potrait of a female necromamcer with big and cute eyes, fine - face, realistic shaded perfect face, fine details. night setting. very anime style. realistic shaded lighting poster by ilya kuvshinov katsuhiro, magali villeneuve, artgerm, jeremy lipkin and michael garmash, rob rey and kentaro miura style, trending on art station'],
       ['steampunk market interior, colorful, 3 d scene, greg rutkowski, zabrocki, karlkka, jayison devadas, trending on artstation, 8 k, ultra wide angle, zenith view, pincushion lens effect'],
       ['“A 

### Only unlock this once the pipeline is ready for 15k prompts at once and you have a place to save the outputs to periodically after every run

In [100]:
# response = anthropic.completions.create(
#         model="claude-2.0",
#         max_tokens_to_sample=3000,
#         prompt=f"""{HUMAN_PROMPT}{prompt_head}{user_prompt}{AI_PROMPT}""",
#         )
# full_response = response.completion

NameError: name 'user_prompt' is not defined